In [5]:
from subprocess import check_output
from pathlib import Path
import os
from subprocess import Popen, PIPE
from getpass import getpass

In [28]:
class Drive():
    mountpoint = None

    def __init__(self, block_file_path,img_size_in_mb=1024*4,sudo_pw=None):
        self.sudoPw = sudo_pw
        if self.sudoPw is None:
            self.sudoPw = getpass("Please input your Sudo password for mount")
        self.block_file_path = block_file_path
        self.img_size_in_mb = img_size_in_mb
        self.debug=False
        if not Path(self.block_file_path).exists():
            self.makeDrive()
            
    def makeDrive(self):
        print(self.block_file_path, self.img_size_in_mb)
        check_output(["/usr/bin/dd", "if=/dev/zero", f"of={self.block_file_path}", "bs=1M", f"count={self.img_size_in_mb}"])
        p = Popen(['sudo', '-S', 'mkfs.ext4',f'-E', f'root_owner={os.getuid()}:{os.getgid()}', f"{self.block_file_path}",f'{self.img_size_in_mb}'], stdin=PIPE, stdout=PIPE, stderr=PIPE)
        stdout, stderr = p.communicate(self.sudoPw.encode('utf-8')+b"\n")
        if self.debug:
            print(stdout,stderr)

        
    def makeMountpoint(self):
        Path(self.mountpoint).mkdir()
        
    def mount(self, mountpoint):
        self.mountpoint = mountpoint
        
        if not Path(self.mountpoint).exists():
            self.makeMountpoint()
        cmd= ['mount', '-o', 'loop,rw,sync', f"{self.block_file_path}",f"{self.mountpoint}"]
        print(' '.join(cmd))
        p = Popen(['sudo', '-S', ]+cmd, stdin=PIPE, stdout=PIPE, stderr=PIPE)
        stdout, stderr = p.communicate(self.sudoPw.encode('utf-8')+b"\n")
        if self.debug:
            print(stdout, stderr)
        
    def dismount(self):
        p = Popen(['sudo', '-S', 'umount','-l', self.mountpoint], stdin=PIPE, stdout=PIPE, stderr=PIPE)
        stdout, stderr = p.communicate(self.sudoPw.encode('utf-8')+b"\n")
        if self.debug:
            print(stdout, stderr)
        self.mountpoint = None
    
    def __repr__(self):
        out = []
        out.append(f"{self.block_file_path}")
        out.append(f' Size: {self.img_size_in_mb} MB')
        out.append(f' MountPoint: {self.mountpoint}')
        return '|'.join(out)

In [29]:
a = Drive('./bobo.drive')
a.debug=True

In [35]:
a.mount('./mountpoint')
a

In [31]:
os.listdir(a.mountpoint)

In [36]:
a.dismount()

In [17]:
a.mountpoint

In [48]:
os.listdir(a.mountpoint)